Steps:
1. read the chapter
2. return character name with profile
3. generate a image for each character (used for consistency across chapters)
4. break the chapter content into screenplay, a collection of scenes
5. generate video for each scene with contextual input.

In [26]:
import boto3
import json
import base64
from PIL import Image
import io
import os
from botocore.config import Config
import random
import time
import uuid
from boto3.dynamodb.conditions import Key
import subprocess
import tempfile
import shutil
import re
from typing import List
import math
from decimal import Decimal
import xml.sax.saxutils as saxutils

### function to get characters introduced in a story chapter

In [2]:
def get_characters_with_visual_profile(chapter_text):

    client = boto3.client("bedrock-runtime")
    system = [{ "text": "You are an expert in analyzing fictional stories" }]

    text = f"""
          Extract the characters introduced in this chapter text. Also provide a brief summary of their most likely visual looks. 
          You can add imaginative visual accentuations to their physical looks so that the readers can perceive it better. The chapter text is:
          {chapter_text}
          \n\n
          The response should only have the output as a pure json object with keys character_name, character_description. NO ADDITIONAL OUTPUT SHOULD BE PRINTED.
          """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.7}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-micro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to read chapter text

In [3]:
def get_story_chapter_text(chapter_file_with_path):
    with open(chapter_file_with_path, 'r') as file:
        chapter_content = file.read()
        return chapter_content

### function to split the story text as per video segment size
#### video segment size is 6 seconds in Nova
#### assume average speaking rate of 150 words per minute in english
#### thus every six seconds can have at most 15 words or lesser

In [4]:
def split_chapter_into_segments(chapter_text):
    # Find the start of the text after "Text:"
    start_index = chapter_text.find("Text:") + len("Text:")
    if start_index == len("Text:"):
        raise ValueError("The input text does not contain 'Text:'")
    
    # Extract the text after "Text:"
    chapter_text = chapter_text[start_index:].strip()
    
    # Split the chapter into sentences using regular expression
    sentences = re.split(r'(?<=[.!?]) +', chapter_text.replace('\n', ' '))
    
    segments = []
    current_segment = ""
    current_word_count = 0
    
    for sentence in sentences:
        sentence_words = sentence.split()
        sentence_word_count = len(sentence_words)
        
        if current_word_count + sentence_word_count > 15:
            # If adding the current sentence exceeds 15 words, start a new segment
            if current_segment:
                segments.append(current_segment.strip())
            current_segment = sentence
            current_word_count = sentence_word_count
        else:
            # Add the sentence to the current segment
            if current_segment:
                current_segment += " " + sentence
            else:
                current_segment = sentence
            current_word_count += sentence_word_count
    
    # Append the last segment
    if current_segment:
        segments.append(current_segment.strip())
    
    return segments

### function to generate image variations using Nova

In [5]:
def generate_image_variation_using_nova(input_image_path, prompt_text, story_so_far, scene_number, output_image_directory):

    def generate_seed():
        return random.randint(10000, 99999)

    model_id = 'amazon.nova-canvas-v1:0'
    input_image_not_found = False
    
    # Check if the input image path exists
    if not os.path.exists(input_image_path):
        input_image = ""
        input_image_not_found = True
    else:
        input_image_not_found = False
        with open(input_image_path, "rb") as image_file:
            input_image = base64.b64encode(image_file.read()).decode('utf8')

    final_prompt_text = f"""
                        Generate a photo-realistic, 8k, HDR image based on the following description: {prompt_text}.
                        
                        Please use the provided context to add detail and context to the output image.
                        
                        Context:
                        {story_so_far}
                        """

    if not input_image_not_found:
        body = json.dumps({
            "taskType": "IMAGE_VARIATION",
            "imageVariationParams": {
                "text": final_prompt_text,
                "negativeText": "bad quality, low resolution, cartoon",
                "images": [input_image] if input_image else [],  # Ensure the list is empty if input_image is blank
                "similarityStrength": 1,
            },
            "imageGenerationConfig": {
                "numberOfImages": 1,
                "height": 720,
                "width": 1280,
                "cfgScale": 8.0,
                "seed": generate_seed()
            }
        })
    elif input_image_not_found:
        body = json.dumps({
            "taskType": "TEXT_IMAGE",
            "textToImageParams": {
                "text": final_prompt_text
            },
            "imageGenerationConfig": {
                "numberOfImages": 1,
                "height": 720,
                "width": 1280,
                "cfgScale": 8.0,
                "seed": generate_seed()
            }
        })

    bedrock = boto3.client(service_name='bedrock-runtime', 
                           config=Config(read_timeout=300))

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )

    response_body = json.loads(response.get("body").read())
    base64_image = response_body.get("images", [])[0] if response_body.get("images") else ""

    if base64_image:
        base64_bytes = base64_image.encode('ascii')
        image_bytes = base64.b64decode(base64_bytes)

        image = Image.open(io.BytesIO(image_bytes))

        image_filename = str(scene_number) + '.jpeg'
        image_path = os.path.join(output_image_directory, image_filename)
        image.save(image_path, 'JPEG')
    else:
        print("No image generated or input image path is invalid.")

### function to generate image using nova

In [6]:
def image_generate_using_nova(character_name, prompt_text, output_directory):

    def format_file_name(name):
        if ' ' not in name:
            return name.lower()
        else:
            return name.lower().replace(' ', '_')

    model_id = 'amazon.nova-canvas-v1:0'
    prompt_text = 'realistic editorial photo for the following profile. ' + prompt_text
    body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt_text
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 720,
            "width": 1280,
            "cfgScale": 8.0,
            "seed": 0
        }
     })

    bedrock = boto3.client(service_name='bedrock-runtime')

    accept = "application/json"
    content_type = "application/json"

    print('generating image for', character_name)

    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, 
                                    contentType=content_type)

    response_body = json.loads(response.get("body").read())

    base64_image = response_body.get("images")[0]
    base64_bytes = base64_image.encode('ascii')
    image_bytes = base64.b64decode(base64_bytes)

    image = Image.open(io.BytesIO(image_bytes))

    image_filename = format_file_name(character_name) + '.jpeg'
    image_path = os.path.join(output_directory, image_filename)
    image.save(image_path, 'JPEG')

### function to get the subject person from the screenplay text

In [7]:
def get_subject_entity(scene_text):

    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                           config=Config(read_timeout=300,
                                         retries=retry_config.retries))

    system = [{"text": "You are an expert in identifying the principal entity or the subject from a given sentence."}]

    text = f"""
            You are an expert in identifying the principal entity or the subject from a given sentence. 
            Based on the sentence below, output ONLY the main subject entity, which MUST be a proper noun (name of a person). 
            Do NOT return common nouns or any other type of words.

            Example:
            "Elena Martin shakes hands with Julien Moreau." -> "Elena Martin"
            "The cat jumps over the fence." -> "None" (since there is no proper noun)
            "Julien Moreau travels to Paris." -> "Julien Moreau"
            "A sketchbook was pulled out by Elena Martin." -> "Elena Martin"

            If there are multiple proper nouns, return the first one encountered.
            If no proper noun is found, return "None".

            The output should ONLY be the main subject entity. NO ADDITIONAL DESCRIPTION or introduction is required.

            Input Text:
            {scene_text}

            Focus on precision, clarity, and visual singularity in scene extraction.
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.5}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate optimized video prompt from scene description

In [8]:
def get_video_generation_prompts(input_text, summary, location_list, chapter_segment):
    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                          config=Config(read_timeout=300, 
                                        retries=retry_config.retries))

    system = [{"text": "You are an expert creating short but concise summaries from screen play text."}]

    text = f"""
            YYou are an expert in semantic analysis and text parsing. Your task is to analyze the input text and extract the following elements: setting, characters, action, and additional details. You may also refer to the overall summary ({summary}) and the exact text from the story ({chapter_segment}) to pick up any missing details whenever required.

            Based on the extracted elements, you will then construct a rephrased text in the format: "setting, characters action, additional details. The scene is captured with a camera technique, highlighting the details of the scene."
            
            Here are the guidelines for choosing the camera technique:
            - If the text describes a specific area or location, choose a camera technique from the following: "drone shot", "slow panning shot", "wide-angle shot".
            - If the text describes a specific human action or interaction, choose a camera technique from the following: "tracking shot", "close-up shot", "over-the-shoulder shot".
            
            If setting is not available, you can refer to the {location_list} referred in the chapter and select the most appropriate setting for the example.
            If any element (setting, characters, action, additional details) is not specified in the input text, infer it from the context provided in the summary or chapter segment. Ensure that the output is clean and free of placeholders or unnecessary text.
            
            Input Text:
            {input_text}
            
            Output Requirement:
            ONLY output the rephrased video generation prompt in the specified format. Do not include any additional text, explanations, or introductions.
            
            Output Format:
            setting, characters action, additional details. The scene is captured with a camera technique, highlighting the details of the scene
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 1000, "topP": 0.2, "temperature": 0.2}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate screen play summary

In [9]:
def get_screen_play_summary(scene_description_master):

    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                          config=Config(read_timeout=300, 
                                        retries=retry_config.retries))

    system = [{"text": "You are an expert creating short but concise summaries from screen play text."}]

    text = f"""
            You are an expert in crafting short, concise summaries from story chapters. 
            Based on the description below, create a 50-word summary.
            DO NOT USE ANY CHARACTER NAMES. THEY SHOULD BE CHANGED WITH PRONOUNS OR COMMON NOUNS.

            The output should ONLY be the summary text. 
            No additional description or introduction is required.

            Input Text:
            {scene_description_master}

            Focus on precision, clarity, and visual singularity in scene extraction.
            Ensure the summary uses extremely neutral language and avoids any emotionally charged or sensational phrases.
            Specifically, avoid phrases like "profound connection," "extraordinary bond," "budding relationship," or any implication of a developing connection.
            Instead, focus purely on the activities or discussions, using factual and neutral terms.
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.5}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to get the location, background, settings

In [10]:
def get_location_background_settings(chapter_content):
    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                          config=Config(read_timeout=300, 
                                        retries=retry_config.retries))

    system = [{"text": "You are an expert creating short but concise summaries from screen play text."}]

    text = f"""
            You are an expert in text analysis and extraction. Your task is to analyze the input text and extract all locations, places, backgrounds, and any location descriptors available. The output should be returned as a comma-separated list in Python style.

            Here are the types of locations to look for:
            - City names (e.g., Paris, Mumbai)
            - Specific places (e.g., coffee shop, office, library, car, park, swimming pool)
            - Any other location descriptors (e.g., "a quaint village", "a bustling market", "sea side", "sunset background", "river side")
            
            Input Text:
            {chapter_content}
            
            Output Requirement:
            ONLY output the extracted locations as a comma-separated list in Python style. Do not include any additional text, explanations, or introductions.
            
            Output Format:
            ["location1", "location2", "location3", ...]

            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 1000, "topP": 0.2, "temperature": 0.2}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to convert story narrative to image prompt

In [11]:
def generate_image_gen_prompt_from_story_narrative(narrative_text, character_list):
    retry_config = Config(retries={'max_attempts': 10, 'mode': 'adaptive'})
    client = boto3.client("bedrock-runtime", 
                                   config=Config(read_timeout=300, retries=retry_config.retries))
    system = [{"text": "You are an expert in converting story narratives to generate prompts in cinematographic style"}]
    text = f"""
        You are an expert visual story analyst specializing in breaking down narrative text into a single, consolidated visual scene.

        Important Rules:
        - If a sentence contains multiple actions, SELECT THE MOST IMPORTANT ACTION FOR OUTPUT
        - Each scene should capture a discrete, singular human movement or interaction, or description
        - If the sentence has an important location, such as a city name, or a cafe, or a theatre, include it in the scene.
        - Avoid compound or complex actions in a single scene description
        - Ensure that each scene description includes the names of the characters involved, if present
        - If no direct human reference is present, focus on converting the sentence into a best-fitting prompt for image generation, describing the place, weather, or other relevant details.
        - Avoid including sensory details such as sounds, smells, or tastes
        - Avoid using pronouns; use character names instead
        - Write each scene description in the passive voice
        - Avoid including dialogue
        - Ensure that the sentence construction is clear and concise
        - NO ADDITIONAL TEXT SHOULD BE PRINTED

        Objective: Transform a narrative text into a precise, singular visual moment, prioritizing human interactions or presence when available. The output should be a single, consolidated description suitable for subsequent image generation.

        Scene Extraction Guidelines:
        1. Isolate and extract the SINGLE, MOST IMPORTANT human action or description
        2. Ensure each scene represents one clear, observable action or description
        3. Maintain chronological sequence of actions
        4. Use the names of the characters in each scene description, if present
        5. If no direct human reference is present, focus on place, weather, or other relevant details for the scene description
        6. Exclude sensory details such as sounds, smells, or tastes
        7. Avoid using pronouns; use character names instead
        8. Write the scene description in the passive voice
        9. Avoid including dialogue
        10. Ensure that the sentence construction is clear and concise

        Examples of Proper Splitting:
        - INCORRECT: "John walked into the room and sat down."
        - CORRECT: 
          DESCRIPTION="John sat down in a chair."

        - INCORRECT: "Their eyes met"
        - CORRECT: "DESCRIPTION="Eyes met between Julien Moreau and Elena Martin.""

        - INCORRECT: "The door chimed again."
        - CORRECT: "DESCRIPTION="Julien Moreau entered the cafe.""

        - INCORRECT: "They spoke of art, literature, their dreams, and their aspirations."
        - CORRECT: "DESCRIPTION="Art, literature, dreams, and aspirations were discussed between Elena Martin and Julien Moreau.""

        - INCORRECT: "The sun set over the horizon, casting a warm glow."
        - CORRECT: "DESCRIPTION="A warm glow was cast by the setting sun over the horizon.""

        - INCORRECT: "Elena’s breath misted in the cool air as she strolled through Montmartre, her eyes drinking in the beauty of the eclectic colors and the life of the city around her."
        - CORRECT: "DESCRIPTION="Elena strolled through Montmartre, observing the eclectic colors and city life.""

        What to Include:
        - Specific character interactions
        - Singular physical actions
        - Clear, observable changes in character position or state
        - Names of the characters involved in each action
        - Descriptions of place, weather, or other relevant details if no human interaction is present

        What to Exclude:
        - Compound actions
        - Internal thoughts
        - Sensory details such as sounds, smells, or tastes
        - Pronouns; use character names instead
        - Dialogue
        - ADDITIONAL TEXT SUCH AS INTRODUCTION, EXPLANATION, ETC.

        Output Requirements:
        - A single, consolidated description in the "DESCRIPTION" format
        - The scene must represent ONE human action or description
        - Visually descriptive
        - Include character names in the scene description, if present
        - Exclude sensory details, pronouns, and dialogue
        - Write the scene description in the passive voice
        - Ensure that the sentence construction is clear and concise

        Input Text:
        {narrative_text}

        Output Requirement:
        Always print the full character name from this dictionary {character_list}

        Output Format:
                DESCRIPTION="Specific single action or description"
        Focus on precision, clarity, and visual singularity in scene extraction.
        """
    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 1000, "topP": 0.2, "temperature": 0.1}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate character visual profile

In [12]:
def generate_character_visual_profile(character_list_collection):

    if character_list_collection is not None:
        image_output_dir = 'pocketfm/character-profile-image'
        character_list_collection_json = json.loads(character_list_collection)
        for character in character_list_collection_json['characters']:
            character_name = character['character_name']
            character_description = character['character_description']
            image_generate_using_nova(character_name, character_description, image_output_dir)

### utility functions

In [13]:
def format_string(input_string):
    lower_case_string = input_string.lower()
    converted_string = lower_case_string.replace(" ", "_")
    return converted_string


def upload_image_to_s3(bucket_name, object_key, file_path):
    s3 = boto3.client('s3')
    response = s3.upload_file(file_path, bucket_name, object_key)
    return response


def update_ddb_record(scene_number, scene_description, chapter_segment, s3_image_path, previous_scene_summary):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    segment_id = str(uuid.uuid4())

    response = table.update_item(
        Key={
            'segment_id': segment_id
        },
        UpdateExpression="set scene_description = :sd, chapter_segment = :cs, s3_image_path = :sip, previous_scene_summary = :ssmry, sequence_number = :sqnum",
        ExpressionAttributeValues={
            ':sd': scene_description,
            ':sip': s3_image_path,
            ':cs': chapter_segment,
            ':ssmry': previous_scene_summary,
            ':sqnum': scene_number
        },
        ReturnValues="UPDATED_NEW"
    )

    return response


def replace_names(scene_description):
    names = {
        "Elena Martin": "the woman",
        "Julien Moreau": "the man",
        "Elena": "the woman",
        "Martin": "the woman",
        "Julien": "the man",
        "Moreau": "the man",
        "Elena's": "the woman's",
        "Julien's": "the man's"}

    for name, replacement in names.items():
        pattern = re.escape(name) + r"('s)?"
        scene_description = re.sub(pattern, 
                                   replacement, 
                                   scene_description, 
                                   flags=re.IGNORECASE)
    return scene_description

### loop through the screen play segments

In [14]:
def get_screen_play_segment_collection(max_sequence=None):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    
    scan_kwargs = {
        'FilterExpression': Key('sequence_number').gte(1),
        'ProjectionExpression': '#segment_id, #scene_description, #s3_image_path, #sequence_number, #chapter_segment, #previous_scene_summary',
        'ExpressionAttributeNames': {
            '#segment_id': 'segment_id',
            '#scene_description': 'scene_description',
            '#s3_image_path': 's3_image_path',
            '#sequence_number': 'sequence_number',
            '#chapter_segment': 'chapter_segment',
            '#previous_scene_summary': 'previous_scene_summary'
        }
    }
    
    if max_sequence:
        scan_kwargs['FilterExpression'] = scan_kwargs['FilterExpression'] & Key('sequence_number').lte(max_sequence)
    
    response = table.scan(**scan_kwargs)
    sorted_records = sorted(
        response.get('Items', []), 
        key=lambda x: int(x['sequence_number'])
    )
    
    return sorted_records

### generate video using image (nova)

In [15]:
def generate_video_using_nova(base_image_with_path, base_prompt, context_prompt, s3_bucket, s3_video_prefix):

    def generate_seed():
        return random.randint(10000, 99999)

    retry_config = Config(retries={'max_attempts': 10, 'mode': 'adaptive'})

    bedrock_runtime = boto3.client("bedrock-runtime", 
                                   config=Config(read_timeout=300, retries=retry_config.retries))

    with open(base_image_with_path, "rb") as f:
        input_image_bytes = f.read()
        input_image_base64 = base64.b64encode(input_image_bytes).decode("utf-8")

    prompt = f"""
             Cinematic Dolly Shot of {base_prompt}
             """
    model_input = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": prompt,
            "images": [
                {
                    "format": "png",
                    "source": {
                        "bytes": input_image_base64
                    }
                }
            ]},
        "videoGenerationConfig": {
            "durationSeconds": 6,
            "fps": 24,
            "dimension": "1280x720",
            "seed": generate_seed()
        },
    }
    s3_uri = f"s3://{s3_bucket}/{s3_video_prefix}"
    invocation = bedrock_runtime.start_async_invoke(modelId="amazon.nova-reel-v1:0", 
                                                    modelInput=model_input,
                                                    outputDataConfig={
                                                        "s3OutputDataConfig": {
                                                            "s3Uri": s3_uri
                                                        }
                                                    },)
    print("Response:")
    print(json.dumps(invocation, indent=2, default=str))

    invocation_arn = invocation["invocationArn"]
    while True:
        invocation = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
        output_s3_bucket_with_prefix = invocation["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
        status = invocation["status"]
        if (status == "Completed"):
            print('video is ready for review...')
            break
        elif (status == "InProgress"):
            print('video generation in progress in the s3 bucket', output_s3_bucket_with_prefix)
            time.sleep(5)
        elif (status == "Failed"):
            print('video generation failed. better luck next time..')
            break

    return output_s3_bucket_with_prefix

### generate visualization for screen play

In [16]:
def generate_visualization_for_screen_play(sequence_number, 
                                           scene_description,
                                           screen_play_summary,
                                           chapter_segment,
                                           character_profile_image_dir, 
                                           s3_prefix, 
                                           scene_cover_image_dir):

    sequence_number = int(sequence_number)
    # get the seed image to choose
    subject_entity = get_subject_entity(scene_description)
    subject_entity_formatted = format_string(subject_entity)
    seed_image_input_with_path = character_profile_image_dir + subject_entity_formatted + '.jpeg'

    # the following line is a hack to circumvent the guardrail in bedrock of a specific person
    scene_description = replace_names(scene_description)
    print(f'processing {sequence_number} ,description {scene_description}, and seed image {seed_image_input_with_path}')
    # generate cover image for this scene
    generate_image_variation_using_nova(seed_image_input_with_path, 
                                        scene_description,
                                        screen_play_summary,
                                        sequence_number,
                                        scene_cover_image_dir)
    # upload the image to a s3 bucket
    s3_object_key = f"{s3_prefix}{subject_entity_formatted}_{sequence_number}.jpeg"
    generated_image_path = scene_cover_image_dir + str(sequence_number) + '.jpeg'
    upload_image_to_s3(s3_bucket, s3_object_key, generated_image_path)
    # save record to DDB
    update_ddb_record(sequence_number, 
                      scene_description, 
                      chapter_segment, 
                      s3_object_key, 
                      screen_play_summary)
    # avoid throtlling
    time.sleep(2)

### Generate video segments from screen play segments

In [17]:
def generate_video_segments_from_screen_play_segments(bucket_name, 
                                                      local_image_directory, 
                                                      context_prompt,
                                                      location_list,
                                                      s3_video_prefix, 
                                                      vid_duration_seconds=6):

    def download_s3_image(bucket_name, s3_image_path, local_directory=None):
        s3_client = boto3.client('s3')
        parts = s3_image_path.split('/')
        s3_key = '/'.join(parts)

        filename = parts[-1]
        local_path = os.path.join(local_directory, filename)

        s3_client.download_file(Bucket=bucket_name, Key=s3_key, Filename=local_path)

        return local_path

    def calculate_segment_times(sequence_number, segment_duration=6):
        start_time = (sequence_number - 1) * segment_duration
        end_time = start_time + segment_duration

        return start_time, end_time

    def update_screen_play_segment_video(segment_id, video_url, start_time_seconds, end_time_seconds):
        dynamodb = boto3.resource('dynamodb')
        table = dynamodb.Table('pocketfm_screen_play_scene_map')
        table.update_item(
            Key={'segment_id': segment_id},
            UpdateExpression='SET video_url = :url, start_time_seconds = :start, end_time_seconds = :end',
            ExpressionAttributeValues={':url': video_url, 
                                       ':start': start_time_seconds, 
                                       ':end': end_time_seconds}
        )

    screen_play_segments = get_screen_play_segment_collection()
    for screen_play_segment in screen_play_segments:
        segment_id = screen_play_segment['segment_id']
        sequence_number = (int(screen_play_segment['sequence_number']))
        if sequence_number >=1:
            scene_description = screen_play_segment['scene_description']
            s3_image_path = screen_play_segment['s3_image_path']
            chapter_segment = screen_play_segment['chapter_segment']
            previous_scene_summary = screen_play_segment['previous_scene_summary']
            sequence_number = str(int(screen_play_segment['sequence_number']))
    
            segment_start_time, segment_end_time = calculate_segment_times(int(sequence_number))
            video_generation_prompt = get_video_generation_prompts(scene_description,
                                                                   previous_scene_summary,
                                                                   location_list,
                                                                   chapter_segment)
    
            print('now processing sequence number', sequence_number, 'video prompt=', video_generation_prompt)
            # ###########################################
            # download scene cover image to local folder
            local_image_path = download_s3_image(bucket_name,
                                                 s3_image_path,
                                                 local_image_directory)
            # invoke video generation and get output video_file
            output_s3_bucket_with_prefix = generate_video_using_nova(local_image_path,
                                                                     scene_description,
                                                                     context_prompt,
                                                                     bucket_name,
                                                                     s3_video_prefix)
            # nova names the output video files as output.mp4 by default
            output_video_file_with_s3_path = output_s3_bucket_with_prefix + '/output.mp4'
            # update ddb with the video location in s3
            update_screen_play_segment_video(segment_id, 
                                             output_video_file_with_s3_path, 
                                             segment_start_time, 
                                             segment_end_time)

### Stitch video segments

In [18]:
def stitch_video_segments(output_path='pocketfm/video-out/consolidated_video.mp4'):
    dynamodb = boto3.resource('dynamodb')
    s3_client = boto3.client('s3')
    temp_dir = tempfile.mkdtemp()
    try:
        table = dynamodb.Table('pocketfm_screen_play_scene_map')
        response = table.scan()
        sorted_records = sorted(response['Items'], 
                                key=lambda x: int(x['sequence_number']))
        video_segments = []
        for record in sorted_records:
            s3_url = record['video_url']
            bucket, key = s3_url.replace('s3://', '').split('/', 1)
            local_filename = os.path.join(temp_dir,
                                          f"segment_{record['sequence_number']}.mp4")
            s3_client.download_file(bucket, key, local_filename)
            video_segments.append(local_filename)
        input_file_path = os.path.join(temp_dir, 'input_list.txt')
        with open(input_file_path, 'w') as f:
            for video in video_segments:
                f.write(f"file '{video}'\n")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        ffmpeg_command = [
            'ffmpeg', 
            '-y',  
            '-loglevel', 'error', 
            '-f', 'concat', 
            '-safe', '0', 
            '-i', input_file_path, 
            '-c', 'copy', 
            output_path
        ]
        subprocess.run(ffmpeg_command, check=True)
        print(f"Consolidated video created: {output_path}")
        return output_path
    finally:
        # Always remove temporary directory
        shutil.rmtree(temp_dir, ignore_errors=True)

### function to generate subtitle script (srt)

In [19]:
def convert_seconds_to_srt_time(seconds):
    """Convert seconds to SRT time format (HH:MM:SS,mmm)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millisecs = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millisecs:03d}"


def generate_srt_file(srt_output_directory):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    response = table.scan()
    sorted_items = sorted(response['Items'],  key=lambda x: int(x['sequence_number']))

    video_srt_file_with_path = os.path.join(srt_output_directory, 'video_subtitles.srt')
    with open(video_srt_file_with_path, 'w', encoding='utf-8') as srt_file:
        subtitle_number = 1
        for item in sorted_items:
            start_time = Decimal(item['start_time_seconds'])
            end_time = Decimal(item['end_time_seconds'])
            text = item['chapter_segment']

            srt_file.write(f"{subtitle_number}\n")
            srt_file.write(f"{convert_seconds_to_srt_time(start_time)} --> {convert_seconds_to_srt_time(end_time)}\n")
            srt_file.write(f"{text}\n\n")

            subtitle_number += 1

    print("Generated SRT file: video_subtitles.srt")

### function to add subtitle to the video

In [20]:
def add_subtitles_to_video(subtitle_file_with_path, base_output_video_with_path):
    output_video_with_subtitles = base_output_video_with_path.replace('.mp4', '_with_subtitles.mp4')
    ffmpeg_command = [
        'ffmpeg', 
        '-y',        # Overwrite output file without asking
        '-loglevel', 'error',  # Suppress verbose output, only show errors
        '-i', base_output_video_with_path,  # Input video
        '-vf', f'subtitles={subtitle_file_with_path}',  # Subtitle filter
        output_video_with_subtitles  # Output video
    ]
    subprocess.run(ffmpeg_command, check=True)
    print(f"Video with subtitles created: {output_video_with_subtitles}")

In [21]:
# def generate_audio():
    
#     polly_client = boto3.client('polly')
    
#     dynamodb = boto3.resource('dynamodb')
#     table = dynamodb.Table('pocketfm_screen_play_scene_map')
#     response = table.scan()
#     sorted_items = sorted(response['Items'],  key=lambda x: int(x['sequence_number']))
#     for item in sorted_items:
#         start_time = Decimal(item['start_time_seconds'])
#         end_time = Decimal(item['end_time_seconds'])
#         duration = int(end_time - start_time)
#         text = item['chapter_segment']

#         ssml_text = ('<speak>'
#                      f'<prosody  amazon:max-duration="{duration}s">'
#                      f'{text}'
#                      '</prosody>'
#                     '</speak>')
#         response = polly_client.synthesize_speech(
#             Text=ssml_text,
#             OutputFormat='mp3',
#             VoiceId='Joanna',
#             TextType='ssml'
#         )
#         with open('pocketfm/audio-output/test.mp3', 'wb') as file:
#             file.write(response['AudioStream'].read())
        

# generate_audio()

### function (experimental) generate audio using polly

In [22]:
def generate_audio_segments_from_screen_play_segments(bucket_name,
                                                      local_audio_directory, 
                                                      s3_audio_prefix,
                                                      audio_duration_seconds=6):
    
    def calculate_segment_times(sequence_number, segment_duration=6):
        start_time = (sequence_number - 1) * segment_duration
        end_time = start_time + segment_duration

        return start_time, end_time

    def update_screen_play_segment_audio(segment_id, audio_url, start_time_seconds, end_time_seconds):
        dynamodb = boto3.resource('dynamodb')
        table = dynamodb.Table('pocketfm_screen_play_scene_map')
        table.update_item(
            Key={'segment_id': segment_id},
            UpdateExpression='SET audio_url = :url, audio_start_time_seconds = :start, audio_end_time_seconds = :end',
            ExpressionAttributeValues={
                ':url': audio_url, 
                ':start': start_time_seconds, 
                ':end': end_time_seconds
            }
        )

    def generate_audio_segment(text, duration, output_path):
        polly_client = boto3.client('polly')
        ssml_text = (
            '<speak>'
            f'<prosody amazon:max-duration="{duration}s">'
            f'{text}'
            '</prosody>'
            '</speak>'
        )
        try:
            response = polly_client.synthesize_speech(
                Text=ssml_text,
                OutputFormat='mp3',
                VoiceId='Joanna',
                TextType='ssml'
            )
            with open(output_path, 'wb') as file:
                file.write(response['AudioStream'].read())
            return True
        except Exception as e:
            print(f"Error generating audio: {e}")
            return False

    def upload_to_s3(local_file_path, bucket_name, s3_prefix):
        s3_client = boto3.client('s3')
        unique_filename = f"{uuid.uuid4()}.mp3"
        s3_key = f"{s3_prefix}/{unique_filename}"
        try:
            s3_client.upload_file(local_file_path, bucket_name, s3_key)
            return f"s3://{bucket_name}/{s3_key}"
        except Exception as e:
            print(f"Error uploading to S3: {e}")
            return None

    def cleanup_temp_audio_files(directory):
        try:
            for filename in os.listdir(directory):
                if filename.endswith('.mp3'):
                    file_path = os.path.join(directory, filename)
                    try:
                        os.remove(file_path)
                        print(f"Deleted temporary file: {file_path}")
                    except Exception as e:
                        print(f"Error deleting {file_path}: {e}")
        except Exception as e:
            print(f"Error during cleanup: {e}")

    os.makedirs(local_audio_directory, exist_ok=True)
    generated_audio_files = []

    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    response = table.scan()
    
    sorted_items = sorted(response['Items'], key=lambda x: int(x['sequence_number']))

    try:
        for item in sorted_items:
            segment_id = item['segment_id']
            sequence_number = int(item['sequence_number'])
            text = item['chapter_segment']
            segment_start_time, segment_end_time = calculate_segment_times(sequence_number)
            duration = int(segment_end_time - segment_start_time)
            local_audio_path = os.path.join(local_audio_directory, f"segment_{sequence_number}.mp3")
            if generate_audio_segment(text, duration, local_audio_path):
                s3_audio_url = upload_to_s3(local_audio_path, bucket_name, s3_audio_prefix)
                if s3_audio_url:
                    update_screen_play_segment_audio(
                        segment_id, 
                        s3_audio_url, 
                        segment_start_time, 
                        segment_end_time
                    )
                    print(f"Processed audio for segment {sequence_number}")
                    generated_audio_files.append(local_audio_path)
    except Exception as e:
        print(f"Error during audio generation: {e}")
    finally:
        cleanup_temp_audio_files(local_audio_directory)

### function to stitch audio file segments

In [23]:
def create_silent_audio(duration, output_path):
    """
    Create a silent audio file of specified duration
    
    :param duration: Duration in seconds
    :param output_path: Path to save the silent audio file
    """
    ffmpeg_cmd = [
        'ffmpeg',
        '-loglevel', 'error',
        '-y',
        '-f', 'lavfi',
        '-i', f'anullsrc=duration={duration}:sample_rate=44100:channel_layout=stereo',
        '-c:a', 'libmp3lame',
        '-b:a', '192k',
        output_path
    ]
    
    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"Created silent audio file: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error creating silent audio: {e}")

def normalize_audio(input_path, output_path):
    """
    Normalize audio file to ensure consistent volume levels
    
    :param input_path: Path to input audio file
    :param output_path: Path to save normalized audio file
    """
    ffmpeg_cmd = [
        'ffmpeg',
        '-loglevel', 'error',
        '-y',
        '-i', input_path,
        '-filter:a', 'loudnorm=I=-16:LRA=11:TP=-1.5',
        '-c:a', 'libmp3lame',
        '-b:a', '192k',
        output_path
    ]
    
    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"Normalized audio: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error normalizing audio: {e}")

def overlay_audio_segment(base_audio_path, segment_audio_path, output_audio_path, start_time):
    """
    Overlay a segment audio file onto a base audio file starting at a specified time
    
    :param base_audio_path: Path to the base audio file
    :param segment_audio_path: Path to the segment audio file
    :param output_audio_path: Path to save the output audio file
    :param start_time: Start time in seconds for the segment
    """
    overlay_time = int(start_time * 1000)  # Convert start time to milliseconds
    
    ffmpeg_cmd = [
        'ffmpeg',
        '-loglevel', 'error',
        '-y',
        '-i', base_audio_path,
        '-i', segment_audio_path,
        '-filter_complex',
        f'[0:a]volume=1[a1];[1:a]volume=1,adelay={overlay_time}|{overlay_time}[a2];[a1][a2]amix=inputs=2:duration=longest:normalize=0[out]',
        '-map', '[out]',
        '-c:a', 'libmp3lame',
        '-b:a', '192k',
        output_audio_path
    ]
    
    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"Overlayed segment onto base audio: {output_audio_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error overlaying audio segment: {e}")

def download_s3_file(s3_url, local_directory):
    """
    Download file from S3 to local directory
    
    :param s3_url: S3 URL of the file
    :param local_directory: Local directory to save the file
    :return: Local file path
    """
    s3_client = boto3.client('s3')
    
    # Parse S3 URL
    bucket = s3_url.split('//')[1].split('/')[0]
    key = '/'.join(s3_url.split('//')[1].split('/')[1:])
    
    # Generate local filename
    filename = os.path.basename(key)
    local_path = os.path.join(local_directory, filename)
    
    # Download file
    s3_client.download_file(bucket, key, local_path)
    
    return local_path

def stitch_audio_segments(output_audio_path, total_video_duration):
    """
    Stitch audio segments with precise timing iteratively
    
    :param output_audio_path: Path to save the final stitched audio
    :param total_video_duration: Total duration of the video in seconds
    """
    # Initialize DynamoDB resource
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    
    # Scan table and sort by sequence number
    response = table.scan()
    sorted_items = sorted(response['Items'], key=lambda x: int(x['sequence_number']))
    
    # Create a temporary directory for downloads and processing
    with tempfile.TemporaryDirectory() as temp_dir:
        # Create master silent audio file
        master_audio_path = os.path.join(temp_dir, 'master_silent_audio.mp3')
        create_silent_audio(total_video_duration, master_audio_path)
        
        current_audio_path = master_audio_path
        
        for index, item in enumerate(sorted_items):
            # Skip items without audio URL
            if 'audio_url' not in item:
                continue
            
            # Download audio file
            audio_url = item['audio_url']
            local_audio_path = download_s3_file(audio_url, temp_dir)
            
            # Normalize the downloaded audio segment
            normalized_audio_path = os.path.join(temp_dir, f'normalized_{index}.mp3')
            normalize_audio(local_audio_path, normalized_audio_path)
            
            # Get segment timing
            start_time = float(item['audio_start_time_seconds'])
            
            # Overlay the segment onto the current audio file
            output_audio_path_temp = os.path.join(temp_dir, f'stitched_audio_{index}.mp3')
            overlay_audio_segment(current_audio_path, normalized_audio_path, output_audio_path_temp, start_time)
            
            # Update the current audio path to the newly created file
            current_audio_path = output_audio_path_temp
        
        # Final normalization of the complete audio
        final_normalized_path = os.path.join(temp_dir, 'final_normalized.mp3')
        normalize_audio(current_audio_path, final_normalized_path)
        
        # Copy the final stitched audio to the desired output path
        shutil.copy2(final_normalized_path, output_audio_path)
        print(f"Final stitched audio created at {output_audio_path}")

# Example usage
if __name__ == "__main__":
    stitch_audio_segments(
        '/home/sagemaker-user/pocketfm/consolidated-audio/consolidated_audio.mp3', 
        total_video_duration=204  # 3 minutes 24 seconds
    )

Created silent audio file: /tmp/tmplpro5kwy/master_silent_audio.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_0.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_0.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_1.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_1.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_2.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_2.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_3.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_3.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_4.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_4.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_5.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_5.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_6.mp3
Overlayed segment onto base audio: /tmp/tmplpro5kwy/stitched_audio_6.mp3
Normalized audio: /tmp/tmplpro5kwy/normalized_7.mp3
Overl

### function to stitch final audio and video

In [24]:
def stitch_final_video_audio(subtitle_output_video_with_path, synthesized_full_audio_with_path):
    """
    Stitch video and audio files together using FFmpeg
    
    :param subtitle_output_video_with_path: Path to the input video file with subtitles
    :param synthesized_full_audio_with_path: Path to the audio file to be merged
    :return: Path to the output video file with merged audio
    """
    # Create output filename by adding '_with_audio' before the extension
    output_path = subtitle_output_video_with_path.rsplit('.', 1)[0] + '_with_audio.mp4'
    
    ffmpeg_cmd = [
        'ffmpeg',
        '-loglevel', 'error',  # Set log level to error only
        '-y',  # Override output file if it exists
        '-i', subtitle_output_video_with_path,  # Input video
        '-i', synthesized_full_audio_with_path,  # Input audio
        '-c:v', 'copy',  # Copy video stream without re-encoding
        '-c:a', 'aac',  # Use AAC codec for audio
        '-strict', 'experimental',  # Allow experimental codecs
        '-map', '0:v:0',  # Map video from first input
        '-map', '1:a:0',  # Map audio from second input
        '-shortest',  # Finish encoding when the shortest input ends
        output_path
    ]
    
    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print(f"Successfully created video with merged audio: {output_path}")
        return output_path
    except subprocess.CalledProcessError as e:
        print(f"Error merging video and audio: {e}")
        return None

## DRIVER BLOCK

In [25]:
character_profile_image_dir = 'pocketfm/character-profile-image/'
scene_cover_image_dir = 'pocketfm/scene-cover-image/'
s3_bucket = 'pocket-fm-poc'
s3_prefix = 'video-generator/key-frame/'
local_image_directory = 'pocketfm/downloaded-scene-cover-image'
context_prompt = ''
s3_video_prefix = 'video-generator/output-video/'
srt_output_directory = 'pocketfm/video-subtitle/'
subtitle_file_with_path = 'pocketfm/video-subtitle/video_subtitles.srt'
base_output_video_with_path = 'pocketfm/video-out/consolidated_video.mp4'
local_audio_directory = 'pocketfm/audio-output'
s3_audio_prefix = 'video-generator/audio-segments'
synthesized_full_audio_with_path = 'pocketfm/consolidated-audio/consolidated_audio.mp3'
subtitle_output_video_with_path = 'pocketfm/video-out/consolidated_video_with_subtitles.mp4'


# # step 1
chapter_content = get_story_chapter_text('pocketfm/story/whispers-of-eternity/chapter-1.txt')

# # step # 2 - get character name and profile
# character_with_profile_collection = get_characters_with_visual_profile(chapter_content)
# generate_character_visual_profile(character_with_profile_collection)

# step 3
# chapter_summary = get_screen_play_summary(chapter_content)

# step 4
# chapter_segments = split_chapter_into_segments(chapter_content)
# character_list = ["Elena Martin", "Julien Moreau"] # remove hardcode
# for i, chapter_segment in enumerate(chapter_segments):
#     result = generate_image_gen_prompt_from_story_narrative(chapter_segment, character_list)
#     image_prompt_text = result.split('=')[1]
#     print('image_prompt_text', image_prompt_text)
#     generate_visualization_for_screen_play(str(i+1), 
#                                            image_prompt_text,
#                                            chapter_summary,
#                                            chapter_segment,
#                                            character_profile_image_dir, 
#                                            s3_prefix, 
#                                            scene_cover_image_dir)
#     time.sleep(5)


# # step # 5
# location_list = get_location_background_settings(chapter_content)

# # step # 6
# generate_video_segments_from_screen_play_segments(s3_bucket, 
#                                                   local_image_directory,
#                                                   context_prompt,
#                                                   location_list,
#                                                   s3_video_prefix)

# # step # 5
# stitch_video_segments()

# # step # 6
# generate_srt_file(srt_output_directory)

# # step # 7
# add_subtitles_to_video(subtitle_file_with_path, base_output_video_with_path)

# # step # 8
# generate_audio_segments_from_screen_play_segments(s3_bucket,
#                                                   local_audio_directory,
#                                                   s3_audio_prefix)

# # step # 9
# stitch_final_video_audio(subtitle_output_video_with_path, synthesized_full_audio_with_path)